# IMDB Sentiment Analysis

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#import Libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [13]:
!pip install --upgrade gensim
!pip install transformers

In [14]:
#reading the csv files
train = pd.read_csv('drive/My Drive/data/Train.csv')
test = pd.read_csv('drive/My Drive/data/Test.csv')
valid = pd.read_csv('drive/My Drive/data/Valid.csv')

In [15]:
train.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [ ]:
print(train.shape)
print(test.shape)
print(valid.shape)

(40000, 2)
(5000, 2)
(5000, 2)


## Text Preprocessing

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
#download nltk resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### To-do:
- remove special characters e.g tags, etc.
- convert text to lowercase
- tokenize the text (split it into individual tokens or words)
- remove stop words
- stemming(removing the suffixes from words to create a stem) & lemmitization(convert word to its base form)

In [4]:
def preprocess_text(text):

    # Remove HTML tags
    text = re.sub('<[^>]*>', '', text)

    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Combine words back into a single string
    preprocessed_text = ' '.join(words)

    return preprocessed_text

In [5]:
#testing the function
preprocess_text("This is Melodie, and she's doing nlp for the first time <tgif>. Hoorahh!!")

'melodie nlp first time hoorahh'

#### Preprocessing our data

In [10]:
train['preprocessed_text'] = train['text'].apply(preprocess_text)
test['preprocessed_text'] = test['text'].apply(preprocess_text)
valid['preprocessed_text'] = valid['text'].apply(preprocess_text)

In [11]:
train.head()

,text,label,preprocessed_text
0,I grew up (b. 1965) watching and loving the Th...,0,grew b watching loving thunderbird mate school...
1,"When I put this movie in my DVD player, and sa...",0,put movie dvd player sat coke chip expectation...
2,Why do people who do not know what a particula...,0,people know particular time past like feel nee...
3,Even though I have great interest in Biblical ...,0,even though great interest biblical movie bore...
4,Im a die hard Dads Army fan and nothing will e...,1,im die hard dad army fan nothing ever change g...


## Vectorization & Models


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

In [ ]:
X_train = train['preprocessed_text']
y_train = train['label']
X_test = test['preprocessed_text']
y_test = test['label']
X_valid = valid['preprocessed_text']
y_valid = valid['label']

#### SVM with Bag of Wordss and TFID Vectorizer

In [ ]:
#we'll create a function that uses SVM to train data with Bag of Words, and TFID vectorization techniques

def train_svm_with_representations(train_data, test_data, representation):
    if representation == 'bow':
        vectorizer = CountVectorizer()
    elif representation == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Invalid representation. Choose 'bow' or 'tfidf'.")

    X_train = vectorizer.fit_transform(train_data)
    X_test = vectorizer.transform(test_data)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred


###### Bag of Words

In [ ]:
%%time
y_pred_bow = train_svm_with_representations(X_train, X_test, 'bow')
accuracy_bow = accuracy_score(y_test, y_pred_bow)
print(accuracy_bow)

0.8804


##### TFID Vectorizer

In [ ]:
%%time
y_pred_tfidf = train_svm_with_representations(X_train, X_test, 'tfidf')
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(accuracy_tfidf)

0.9004


#### SVM with Custom Word2Vec Vectorizer

In [ ]:
def get_word2vec_embeddings(data):
    tokenized_sentences = [sentence.split() for sentence in data]
    model = Word2Vec(tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
    embeddings = np.array([np.mean([model.wv[word] for word in sentence], axis=0) for sentence in tokenized_sentences])

    return embeddings

def train_svm_with_word2vec(train_data, test_data):
    X_train = get_word2vec_embeddings(train_data)
    X_test = get_word2vec_embeddings(test_data)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred

In [ ]:
%%time
y_pred_word2vec = train_svm_with_word2vec(X_train, X_test)
accuracy_word2vec = accuracy_score(y_test, y_pred_word2vec)
print(accuracy_word2vec)

0.5262
CPU times: user 3min 46s, sys: 1.14 s, total: 3min 47s
Wall time: 3min 22s


#### SVM with Pretrained Google News 300 Word2Vec Vectorizer

In [ ]:
import gensim.downloader as api

def get_google_word2vec_embeddings(data):
    # Load the Google News Word2Vec model
    model = api.load("word2vec-google-news-300")

    tokenized_sentences = [sentence.split() for sentence in data]
    embeddings = []

    for sentence in tokenized_sentences:
        sentence_embeddings = []
        for word in sentence:
            if word in model:
                sentence_embeddings.append(model[word])
        if sentence_embeddings:
            embeddings.append(np.mean(sentence_embeddings, axis=0))
        else:
            embeddings.append(np.zeros(300))

    return np.array(embeddings)

def train_svm_with_google_word2vec(train_data, test_data):
    X_train = get_google_word2vec_embeddings(train_data)
    X_test = get_google_word2vec_embeddings(test_data)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    return y_pred


In [ ]:
y_pred_google_word2vec = train_svm_with_google_word2vec(X_train, X_test)
accuracy_google_word2vec = accuracy_score(y_test, y_pred_google_word2vec)
print(accuracy_google_word2vec)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
0.8574


####  Recurrent Neural Networks

##### Vanilla RNN

Using preprocessed text

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Tokenize the data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['preprocessed_text'])

In [ ]:
X_train = tokenizer.texts_to_sequences(train['preprocessed_text'])
X_test = tokenizer.texts_to_sequences(test['preprocessed_text'])
X_valid = tokenizer.texts_to_sequences(valid['preprocessed_text'])

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_valid = pad_sequences(X_valid, maxlen=max_len)

y_train = to_categorical(train['label'])
y_test = to_categorical(test['label'])
y_valid = to_categorical(valid['label'])

In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25, batch_size=128)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/25
313/313 [==============================] - 17s 51ms/step - loss: 0.4201 - accuracy: 0.7927 - val_loss: 0.3188 - val_accuracy: 0.8602
Epoch 2/25
313/313 [==============================] - 15s 47ms/step - loss: 0.2340 - accuracy: 0.9086 - val_loss: 0.3566 - val_accuracy: 0.8594
Epoch 3/25
313/313 [==============================] - 15s 47ms/step - loss: 0.1250 - accuracy: 0.9560 - val_loss: 0.4319 - val_accuracy: 0.8616
Epoch 4/25
313/313 [==============================] - 15s 47ms/step - loss: 0.0410 - accuracy: 0.9869 - val_loss: 0.5428 - val_accuracy: 0.8532
Epoch 5/25
313/313 [==============================] - 15s 47ms/step - loss: 0.0184 - accuracy: 0.9948 - val_loss: 0.6301 - val_accuracy: 0.8526
Epoch 6/25
313/313 [==============================] - 16s 50ms/step - loss: 0.0132 - accuracy: 0.9963 - val_loss: 0.7404 - val_accuracy: 0.8538
Epoch 7/25
313/313 [==============================] - 15s 47ms/step - loss: 0.0159 - accuracy: 0.9947 - val_loss: 0.7175 - val_accuracy:

Using raw text

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['text'])

X_train1 = tokenizer.texts_to_sequences(train['text'])
X_test1 = tokenizer.texts_to_sequences(test['text'])
X_valid1 = tokenizer.texts_to_sequences(valid['text'])

X_train1 = pad_sequences(X_train1, maxlen=max_len)
X_test1 = pad_sequences(X_test1, maxlen=max_len)
X_valid1 = pad_sequences(X_valid1, maxlen=max_len)

y_train1 = to_categorical(train['label'])
y_test1 = to_categorical(test['label'])
y_valid1 = to_categorical(valid['label'])

# Build the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SimpleRNN(64))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Train the model
history = model.fit(X_train1, y_train1, validation_data=(X_valid1, y_valid1), epochs=25, batch_size=128,
                    callbacks=[early_stop])

# Evaluate the model
loss, accuracy = model.evaluate(X_test1, y_test1)

print("Test accuracy:", accuracy)

Epoch 1/25
313/313 [==============================] - 19s 57ms/step - loss: 0.4208 - accuracy: 0.8012 - val_loss: 0.3628 - val_accuracy: 0.8504
Epoch 2/25
313/313 [==============================] - 20s 64ms/step - loss: 0.2561 - accuracy: 0.8978 - val_loss: 0.3550 - val_accuracy: 0.8566
Epoch 3/25
313/313 [==============================] - 19s 62ms/step - loss: 0.1385 - accuracy: 0.9491 - val_loss: 0.4932 - val_accuracy: 0.8512
Epoch 4/25
313/313 [==============================] - 17s 54ms/step - loss: 0.0562 - accuracy: 0.9818 - val_loss: 0.5796 - val_accuracy: 0.8386
Epoch 5/25
313/313 [==============================] - 18s 57ms/step - loss: 0.0211 - accuracy: 0.9939 - val_loss: 0.6985 - val_accuracy: 0.8300
Epoch 6/25
313/313 [==============================] - 21s 68ms/step - loss: 0.0140 - accuracy: 0.9962 - val_loss: 0.7431 - val_accuracy: 0.8290
Epoch 7/25
313/313 [==============================] - 15s 48ms/step - loss: 0.0260 - accuracy: 0.9909 - val_loss: 0.7721 - val_accuracy:

##### LSTM (Long Short Term Memory) RNN

Using Preprocessed text

In [ ]:
from tensorflow.keras.layers import LSTM

In [ ]:
# Build the model
lstm = Sequential()
lstm.add(Embedding(max_words, 128, input_length=max_len))
lstm.add(LSTM(64))
lstm.add(Dense(2, activation='softmax'))

# Compile the model
lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = lstm.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25, batch_size=128,
                    callbacks=[early_stop])

Epoch 1/25
313/313 [==============================] - 62s 193ms/step - loss: 0.3467 - accuracy: 0.8438 - val_loss: 0.2971 - val_accuracy: 0.8788
Epoch 2/25
313/313 [==============================] - 53s 170ms/step - loss: 0.2199 - accuracy: 0.9136 - val_loss: 0.3009 - val_accuracy: 0.8836
Epoch 3/25
313/313 [==============================] - 59s 188ms/step - loss: 0.1631 - accuracy: 0.9386 - val_loss: 0.3213 - val_accuracy: 0.8786
Epoch 4/25
313/313 [==============================] - 48s 153ms/step - loss: 0.1237 - accuracy: 0.9532 - val_loss: 0.4098 - val_accuracy: 0.8734
Epoch 5/25
313/313 [==============================] - 48s 154ms/step - loss: 0.0903 - accuracy: 0.9669 - val_loss: 0.4451 - val_accuracy: 0.8642
Epoch 6/25
313/313 [==============================] - 47s 151ms/step - loss: 0.0704 - accuracy: 0.9751 - val_loss: 0.5188 - val_accuracy: 0.8622
Epoch 7/25
313/313 [==============================] - 48s 154ms/step - loss: 0.0568 - accuracy: 0.9812 - val_loss: 0.5862 - val_ac

In [ ]:
# Evaluate the model
loss, accuracy = lstm.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

157/157 [==============================] - 2s 12ms/step - loss: 0.2973 - accuracy: 0.8790
Test accuracy: 0.8790000081062317


Using Raw text

In [ ]:
# Build the model
lstm = Sequential()
lstm.add(Embedding(max_words, 128, input_length=max_len))
lstm.add(LSTM(64))
model.add(Dropout(0.5))
lstm.add(Dense(2, activation='softmax'))

# Compile the model
lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = lstm.fit(X_train1, y_train1, validation_data=(X_valid1, y_valid1), epochs=25, batch_size=128,
                    callbacks=[early_stop])

# Evaluate the model
loss, accuracy = lstm.evaluate(X_test1, y_test1)

print("Test accuracy:", accuracy)

Epoch 1/25
313/313 [==============================] - 84s 263ms/step - loss: 0.3864 - accuracy: 0.8241 - val_loss: 0.3120 - val_accuracy: 0.8656
Epoch 2/25
313/313 [==============================] - 60s 191ms/step - loss: 0.2512 - accuracy: 0.8988 - val_loss: 0.3194 - val_accuracy: 0.8616
Epoch 3/25
313/313 [==============================] - 50s 160ms/step - loss: 0.1925 - accuracy: 0.9244 - val_loss: 0.4433 - val_accuracy: 0.8458
Epoch 4/25
313/313 [==============================] - 66s 209ms/step - loss: 0.1448 - accuracy: 0.9453 - val_loss: 0.4204 - val_accuracy: 0.8474
Epoch 5/25
313/313 [==============================] - 62s 197ms/step - loss: 0.1090 - accuracy: 0.9601 - val_loss: 0.4624 - val_accuracy: 0.8484
Epoch 6/25
313/313 [==============================] - 54s 173ms/step - loss: 0.0793 - accuracy: 0.9726 - val_loss: 0.6078 - val_accuracy: 0.8456
Epoch 7/25
313/313 [==============================] - 51s 164ms/step - loss: 0.0592 - accuracy: 0.9800 - val_loss: 0.6642 - val_ac

##### GRU (Gated Recurrent Unit) RNN

Using preprocessed text

In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
# Build the model
gru = Sequential()
gru.add(Embedding(max_words, 128, input_length=max_len))
gru.add(GRU(64))
gru.add(Dense(2, activation='softmax'))

# Compile the model
gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Fit the model
history = gru.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25, batch_size=128,
                    callbacks=[early_stop])

# Evaluate the model
loss, accuracy = gru.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/25
313/313 [==============================] - 47s 145ms/step - loss: 0.3675 - accuracy: 0.8300 - val_loss: 0.2968 - val_accuracy: 0.8778
Epoch 2/25
313/313 [==============================] - 49s 158ms/step - loss: 0.2265 - accuracy: 0.9105 - val_loss: 0.3043 - val_accuracy: 0.8726
Epoch 3/25
313/313 [==============================] - 43s 138ms/step - loss: 0.1741 - accuracy: 0.9340 - val_loss: 0.3223 - val_accuracy: 0.8726
Epoch 4/25
313/313 [==============================] - 45s 145ms/step - loss: 0.1325 - accuracy: 0.9506 - val_loss: 0.4476 - val_accuracy: 0.8624
Epoch 5/25
313/313 [==============================] - 43s 137ms/step - loss: 0.1025 - accuracy: 0.9638 - val_loss: 0.4619 - val_accuracy: 0.8632
Epoch 6/25
313/313 [==============================] - 45s 145ms/step - loss: 0.0798 - accuracy: 0.9722 - val_loss: 0.5101 - val_accuracy: 0.8648
Epoch 7/25
313/313 [==============================] - 43s 139ms/step - loss: 0.0627 - accuracy: 0.9788 - val_loss: 0.5716 - val_ac

Using raw text

In [ ]:
# Build the model
gru = Sequential()
gru.add(Embedding(max_words, 128, input_length=max_len))
gru.add(GRU(64))
gru.add(Dropout(0.5))
gru.add(Dense(2, activation='softmax'))

# Compile the model
gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = gru.fit(X_train1, y_train1, validation_data=(X_valid1, y_valid1), epochs=25, batch_size=128,
                    callbacks=[early_stop])

# Evaluate the model
loss, accuracy = gru.evaluate(X_test1, y_test1)

print("Test accuracy:", accuracy)


Epoch 1/25
313/313 [==============================] - 57s 176ms/step - loss: 0.4234 - accuracy: 0.7985 - val_loss: 0.3304 - val_accuracy: 0.8554
Epoch 2/25
313/313 [==============================] - 50s 159ms/step - loss: 0.2670 - accuracy: 0.8917 - val_loss: 0.3141 - val_accuracy: 0.8670
Epoch 3/25
313/313 [==============================] - 47s 150ms/step - loss: 0.2139 - accuracy: 0.9166 - val_loss: 0.3709 - val_accuracy: 0.8604
Epoch 4/25
313/313 [==============================] - 46s 147ms/step - loss: 0.1624 - accuracy: 0.9394 - val_loss: 0.3618 - val_accuracy: 0.8574
Epoch 5/25
313/313 [==============================] - 44s 142ms/step - loss: 0.1257 - accuracy: 0.9547 - val_loss: 0.4554 - val_accuracy: 0.8518
Epoch 6/25
313/313 [==============================] - 46s 145ms/step - loss: 0.0967 - accuracy: 0.9662 - val_loss: 0.5291 - val_accuracy: 0.8522
Epoch 7/25
313/313 [==============================] - 44s 140ms/step - loss: 0.0779 - accuracy: 0.9739 - val_loss: 0.5542 - val_ac

##### Bi-Directional LSTM RNN

Using preprocessed text

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional

In [ ]:
# Build the model
bi_lstm = Sequential()
bi_lstm.add(Embedding(max_words, 128, input_length=max_len))
bi_lstm.add(Bidirectional(LSTM(64)))
bi_lstm.add(Dense(2, activation='softmax'))

# Compile the model
bi_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = bi_lstm.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=25, batch_size=128,
                    callbacks=[early_stop])

# Evaluate the model
loss, accuracy = bi_lstm.evaluate(X_test, y_test)

print("Test accuracy:", accuracy)

Epoch 1/25
313/313 [==============================] - 92s 281ms/step - loss: 0.3508 - accuracy: 0.8408 - val_loss: 0.2842 - val_accuracy: 0.8848
Epoch 2/25
313/313 [==============================] - 87s 277ms/step - loss: 0.2189 - accuracy: 0.9141 - val_loss: 0.3144 - val_accuracy: 0.8786
Epoch 3/25
313/313 [==============================] - 88s 282ms/step - loss: 0.1610 - accuracy: 0.9387 - val_loss: 0.3607 - val_accuracy: 0.8798
Epoch 4/25
313/313 [==============================] - 86s 274ms/step - loss: 0.1094 - accuracy: 0.9596 - val_loss: 0.3881 - val_accuracy: 0.8626
Epoch 5/25
313/313 [==============================] - 85s 271ms/step - loss: 0.0750 - accuracy: 0.9740 - val_loss: 0.5139 - val_accuracy: 0.8684
Epoch 6/25
313/313 [==============================] - 86s 276ms/step - loss: 0.0508 - accuracy: 0.9835 - val_loss: 0.5908 - val_accuracy: 0.8604
Epoch 7/25
313/313 [==============================] - 87s 279ms/step - loss: 0.0371 - accuracy: 0.9876 - val_loss: 0.6815 - val_ac

Using raw text

In [33]:
# Build the model
bi_lstm = Sequential()
bi_lstm.add(Embedding(max_words, 128, input_length=max_len))
bi_lstm.add(GRU(64))
bi_lstm.add(Dropout(0.5))
bi_lstm.add(Dense(2, activation='softmax'))

# Compile the model
bi_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = bi_lstm.fit(X_train1, y_train1, validation_data=(X_valid1, y_valid1), epochs=25, batch_size=128,
                    callbacks=[early_stop])

# Evaluate the model
loss, accuracy = bi_lstm.evaluate(X_test1, y_test1)

print("Test accuracy:", accuracy)


Epoch 1/25
313/313 [==============================] - 46s 140ms/step - loss: 0.4163 - accuracy: 0.7997 - val_loss: 0.3160 - val_accuracy: 0.8664
Epoch 2/25
313/313 [==============================] - 53s 169ms/step - loss: 0.2646 - accuracy: 0.8938 - val_loss: 0.3127 - val_accuracy: 0.8684
Epoch 3/25
313/313 [==============================] - 77s 247ms/step - loss: 0.2120 - accuracy: 0.9168 - val_loss: 0.3692 - val_accuracy: 0.8582
Epoch 4/25
313/313 [==============================] - 55s 175ms/step - loss: 0.1622 - accuracy: 0.9392 - val_loss: 0.3925 - val_accuracy: 0.8620
Epoch 5/25
313/313 [==============================] - 64s 203ms/step - loss: 0.1195 - accuracy: 0.9567 - val_loss: 0.4201 - val_accuracy: 0.8556
Epoch 6/25
313/313 [==============================] - 44s 141ms/step - loss: 0.0924 - accuracy: 0.9679 - val_loss: 0.4852 - val_accuracy: 0.8552
Epoch 7/25
313/313 [==============================] - 46s 146ms/step - loss: 0.0697 - accuracy: 0.9771 - val_loss: 0.5656 - val_ac

#### Transformer Models

##### Pre-Trained DistilBERT Model

Using preprocessed text

In [8]:
import os
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer


# Detect TPU and set up distribution strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(tpu)

model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
#prepare datasets
max_len = 512

X_train = tokenizer(text=train['preprocessed_text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_train = train['label'].values
X_test = tokenizer(text=test['preprocessed_text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_test = test['label'].values
X_valid = tokenizer(text=valid['preprocessed_text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_valid = valid['label'].values

X_train_np = X_train['input_ids'].numpy()
X_valid_np = X_valid['input_ids'].numpy()

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test['input_ids'], X_test['attention_mask'])).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_valid_np, y_valid)).batch(32)


In [17]:
with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Set up early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

batch_size = 32
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[early_stop])

y_pred = model.predict(test_dataset)
y_pred_labels = np.argmax(y_pred.logits, axis=1)
accuracy = np.sum(y_pred_labels == y_test) / len(y_test)
print(f"Test accuracy: {accuracy:.2f}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/10
1250/1250 [==============================] - 210s 122ms/step - loss: 0.6949 - accuracy: 0.5016 - val_loss: 0.6932 - val_accuracy: 0.5016
Epoch 2/10
1250/1250 [==============================] - 119s 95ms/step - loss: 0.5808 - accuracy: 0.6417 - val_loss: 0.3965 - val_accuracy: 0.8130
Epoch 3/10
1250/1250 [==============================] - 117s 94ms/step - loss: 0.3125 - accuracy: 0.8721 - val_loss: 0.2939 - val_accuracy: 0.8812
Epoch 4/10
1250/1250 [==============================] - 116s 93ms/step - loss: 0.2218 - accuracy: 0.9150 - val_loss: 0.3084 - val_accuracy: 0.8822
Epoch 5/10
1250/1250 [==============================] - 117s 93ms/step - loss: 0.1617 - accuracy: 0.9420 - val_loss: 0.4406 - val_accuracy: 0.8570
Epoch 6/10
157/157 [==============================] - 14s 69ms/step
Test accuracy: 0.88


Using raw text

In [13]:
#prepare datasets
max_len = 512

X_train1 = tokenizer(text=train['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_train1 = train['label'].values
X_test1 = tokenizer(text=test['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_test1 = test['label'].values
X_valid1 = tokenizer(text=valid['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_valid1 = valid['label'].values

X_train_np1 = X_train1['input_ids'].numpy()
X_valid_np1 = X_valid1['input_ids'].numpy()

# Create TensorFlow datasets
train_dataset1 = tf.data.Dataset.from_tensor_slices((X_train_np1, y_train1)).batch(32)
test_dataset1 = tf.data.Dataset.from_tensor_slices((X_test1['input_ids'], X_test1['attention_mask'])).batch(32)
val_dataset1 = tf.data.Dataset.from_tensor_slices((X_valid_np1, y_valid1)).batch(32)


In [14]:
with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Set up early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

batch_size = 32
epochs = 10
history = model.fit(train_dataset1, epochs=epochs, validation_data=val_dataset1, callbacks=[early_stop])

y_pred = model.predict(test_dataset1)
y_pred_labels = np.argmax(y_pred.logits, axis=1)
accuracy = np.sum(y_pred_labels == y_test1) / len(y_test1)
print(f"Test accuracy: {accuracy:.2f}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/10
1250/1250 [==============================] - 150s 98ms/step - loss: 0.2467 - accuracy: 0.8975 - val_loss: 0.1930 - val_accuracy: 0.9228
Epoch 2/10
1250/1250 [==============================] - 118s 94ms/step - loss: 0.1340 - accuracy: 0.9513 - val_loss: 0.2530 - val_accuracy: 0.9068
Epoch 3/10
1250/1250 [==============================] - 117s 94ms/step - loss: 0.0807 - accuracy: 0.9711 - val_loss: 0.2282 - val_accuracy: 0.9226
Epoch 4/10
157/157 [==============================] - 9s 33ms/step
Test accuracy: 0.93


##### Pre-Trained Roberta Model

For this we'll just use raw text since Neural Networks tend to perform best with the raw text.

In [9]:
model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
#prepare datasets
max_len = 512

X_train1 = tokenizer(text=train['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_train1 = train['label'].values
X_test1 = tokenizer(text=test['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_test1 = test['label'].values
X_valid1 = tokenizer(text=valid['text'].tolist(), add_special_tokens=True, max_length=max_len, truncation=True, padding=True, return_tensors='tf')
y_valid1 = valid['label'].values

X_train_np1 = X_train1['input_ids'].numpy()
X_valid_np1 = X_valid1['input_ids'].numpy()

# Create TensorFlow datasets
train_dataset1 = tf.data.Dataset.from_tensor_slices((X_train_np1, y_train1)).batch(16)
test_dataset1 = tf.data.Dataset.from_tensor_slices((X_test1['input_ids'], X_test1['attention_mask'])).batch(16)
val_dataset1 = tf.data.Dataset.from_tensor_slices((X_valid_np1, y_valid1)).batch(16)

In [17]:
with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-6)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Set up early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

batch_size = 16
epochs = 10
history = model.fit(train_dataset1, epochs=epochs, validation_data=val_dataset1, callbacks=[early_stop])

y_pred = model.predict(test_dataset1)
y_pred_labels = np.argmax(y_pred.logits, axis=1)
accuracy = np.sum(y_pred_labels == y_test1) / len(y_test1)
print(f"Test accuracy: {accuracy:.2f}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

Epoch 1/10
2500/2500 [==============================] - 426s 128ms/step - loss: 0.2845 - accuracy: 0.8570 - val_loss: 0.1561 - val_accuracy: 0.9418
Epoch 2/10
2500/2500 [==============================] - 309s 124ms/step - loss: 0.1405 - accuracy: 0.9488 - val_loss: 0.1415 - val_accuracy: 0.9522
Epoch 3/10
2500/2500 [==============================] - 310s 124ms/step - loss: 0.1174 - accuracy: 0.9586 - val_loss: 0.1341 - val_accuracy: 0.9542
Epoch 4/10
2500/2500 [==============================] - 310s 124ms/step - loss: 0.0961 - accuracy: 0.9670 - val_loss: 0.1360 - val_accuracy: 0.9542
Epoch 5/10
2500/2500 [==============================] - 308s 123ms/step - loss: 0.0813 - accuracy: 0.9728 - val_loss: 0.1363 - val_accuracy: 0.9570
Epoch 6/10
313/313 [==============================] - 17s 36ms/step
Test accuracy: 0.95


#### Comparing the Accuracy of all the models for our Sentiment Analysis:
- Accuracy of SVM with BOW: 88.04%
- Accuracy of SVM with Tf-IDF: 90.04%
- Accuracy of SVM with Custom Word2Vec: 52.62%
- Accuracy of SVM with Google Word2Vec: 85.74%
- Accuracy of Vanilla RNN Model: 85.51%
- Accuracy of LSTM Model: 87.90%
- Accuracy of GRU Model: 88.00%
- Accuracy of Bi-Directional LSTM Model: 88.16%
- Accuracy of DistilBERT Model: 93%
- Accuracy of Roberta Model: 95%